In [1]:
import pysam
from collections import defaultdict
import numpy as np

In [3]:
# functions

def read_fasta(infile_path):
    seq_dict = {}
    with open(infile_path,'rU') as file:
        for line in file:
            if line.startswith('>'):
                header = line.strip().replace('>','')
            else:
                seq_dict[header] = line.strip()
    return seq_dict


## Do per-base average accuracy for VI41 reads (prior to nanopolishing)

The VI41 BAM file has 448,035 reads assembled to reference KJ776791.2, which is a Zika genome sampled from French Polynesia on 28-Nov-2013, and sequenced on IonTorrent. VI41 was sequenced using ONT 1D ligation sequencing, and the consensus genome was generated using our `zika-seq` pipeline, including using Nanopolish.

In [2]:

## note that length of the dict won't be the same as the length of the reference, because only sites with bases from read and reference are counted
## IE if at one site all the reads have been softclipped, that won't get recorded in the dict

compositionVI41 = defaultdict(dict) #NESTED DICT where key is index of reference, value is a dict with counts of A,C,G,T for all reads at that site.
depthVI41 = defaultdict(int)
bamVI41 = pysam.AlignmentFile("VI41.trimmed.sorted.bam", "rb")

for read in bamVI41:
    for pair in read.aligned_pairs: #aligned_pairs is a tuple with where (index of read, index of reference) shows how read and reference align.
        if pair[0] is not None and pair[1] is not None: #none indicates that this portion did not map to reference, and is soft trimmed in the bamfile.
            depthVI41[pair[1]] += 1 #count that you have one read's coverage at that site
            base_composition = compositionVI41[pair[1]] #stub out empty dict that is the value associated with the composition dict key (the reference index)
            query_base = read.query_sequence[pair[0]] #what base is at this site for this read?
            if not query_base in compositionVI41[pair[1]]:
                compositionVI41[pair[1]][query_base] = 0 #add that base to the site i dict if you haven't seen the base before in any other read
            compositionVI41[pair[1]][query_base] += 1 #count the occurrence of seeing that base at site i

### To avoid any issues of adding keys to dict by querying a key that doesn't exist
### convert default dicts to normal dicts.

final_depthVI41 = dict(depthVI41)
final_compositionVI41 = dict(compositionVI41)

In [6]:
print 'There are {} sites with read pileups present in the BAM file.'.format(len(final_compositionVI41)) #how many keys are in here?
assert len(final_depthVI41) == len(final_compositionVI41)

There are 10192 sites with read pileups present in the BAM file.


In [7]:
# import reference sequence 
with open('/Users/alliblk/Desktop/gitrepos/zika-seq/seq-validation/KJ776791.2.fasta','rU') as file:
    for line in file:
        if line.startswith('>'):
            continue
        else: 
            reference = line.strip()

In [8]:
# import consensus sequence for VI41
with open('/Users/alliblk/Desktop/gitrepos/zika-seq/seq-validation/VI41.consensus.polished.fasta', 'rU') as file:
    for line in file:
        if line.startswith('>'):
            continue
        else: 
            consensusVI41 = line.strip()


In [13]:
# Calculate per base accuracy stats for VI41
per_base_accuraciesVI41= []

for i in range(len(consensusVI41)):
    correct_base = consensusVI41[i]
    if correct_base != 'N': #don't look at sites that were masked due to coverage issues, because we don't care about sites we don't actually call
        proportion_reads_correct = float(final_compositionVI41[i][correct_base])/final_depthVI41[i] #do not need to worry about key errors because any sites without a BAM stack will have been called with 'N'
        per_base_accuraciesVI41.append(proportion_reads_correct)
    elif correct_base == 'N':
        per_base_accuraciesVI41.append(float('NaN')) # do not calculate for sites we do not call.


print "The mean per-base sequencing accuracy is {}".format(np.nanmean(per_base_accuraciesVI41))
print "The standard deviation is {}".format(np.nanstd(per_base_accuraciesVI41))
print '\n'
print "The following 0-indexed sites in the genome had sequencing accuracies less than 50%, and warrant some investigation into what's going on: {}".format([index for index,value in enumerate(per_base_accuraciesVI41) if value < 0.5])

### This is interesting. This is a place where the majority consensus would have called a SNP, but nanopolish returned back to reference base.
### Looking into this a bit more deeply in the cell below.


The mean per-base sequencing accuracy is 0.962493244973
The standard deviation is 0.0373886072273


The following 0-indexed sites in the genome had sequencing accuracies less than 50%, and warrant some investigation into what's going on: [5314, 10391]


In [24]:
# Looking into some surprising places where BAM stack supported a variant (and direct consensus generation from BAM featured variant)
# but where nanopolish reverted the site back to the same base as is present in the reference genome.

VI41_dict = read_fasta('/Users/alliblk/Desktop/gitrepos/zika-seq/seq-validation/VI41-ref-nanopolish-comp-aligned.fasta')
VI41_ref = VI41_dict['KJ776791.2_reference']
VI41_polished = VI41_dict['VI41_albacore_polished']
VI41_unpolished = VI41_dict['VI41_not_polished']

#check that sites with seq-accuracy < 0.5 in fact show different bases between the BAM-generated consensus and the nanopolished consensus.
index = 0
for (a,b) in zip(VI41_polished,VI41_unpolished):
    if a != b and a!='n' and b != 'n' and a!='-' and b != '-':
        print "The polished and BAM-generated consensus genomes disagreed at 0-indexed site {}.".format(index)
        print "The polished genome has a {}, and the BAM-generated genome has a {}".format(a.upper(),b.upper())
    index += 1
    
print '\n'
print 'Details about 0-indexed site 5314'
print 'The sequencing accuracy at this site was {}.'.format(per_base_accuraciesVI41[5314])
print 'Reference: {}'.format(reference[5314])
print 'Polished: {}'.format(VI41_polished[5314].upper())
print 'Unpolished: {}'.format(VI41_unpolished[5314].upper())
print 'Read counts for each base from the read pileup at this site: {}.'.format(final_compositionVI41[5314])
print '\n'
print 'Details about 0-indexed site 10391'
print 'The sequencing accuracy at this site was {}.'.format(per_base_accuraciesVI41[10391])
print 'Reference: {}'.format(reference[10391])
print 'Polished: {}'.format(VI41_polished[10391].upper())
print 'Unpolished: {}'.format(VI41_unpolished[10391].upper())
print 'Read counts for each base from the read pileup at this site: {}.'.format(final_compositionVI41[10391])

The polished and BAM-generated consensus genomes disagreed at 0-indexed site 5314.
The polished genome has a A, and the BAM-generated genome has a G
The polished and BAM-generated consensus genomes disagreed at 0-indexed site 10391.
The polished genome has a G, and the BAM-generated genome has a A


Details about 0-indexed site 5314
The sequencing accuracy at this site was 0.249393605292.
Reference: A
Polished: A
Unpolished: G
Read counts for each base from the read pileup at this site: {'A': 1131, 'C': 1, 'T': 4, 'G': 3399}.


Details about 0-indexed site 10391
The sequencing accuracy at this site was 0.0474751834268.
Reference: G
Polished: G
Unpolished: A
Read counts for each base from the read pileup at this site: {'A': 4156, 'C': 183, 'T': 75, 'G': 220}.


## Per-base and average sequencing accuracy for VI42 reads (prior to nanopolishing)

The VI42 BAM file has 48,367 reads assembled to reference KJ776791.2, which is a Zika genome sampled from French Polynesia on 28-Nov-2013, and sequenced on IonTorrent. VI42 was sequenced using ONT 1D ligation sequencing, and the consensus genome was generated using our `zika-seq` pipeline, including using Nanopolish.

In [25]:

depthVI42 = defaultdict(int) #dict where key is index of the reference sequence (0 to 10808), value is total sum of all reads covering that site.
compositionVI42 = defaultdict(dict)#NESTED DICT where key is index of reference, value is a dict with counts of A,C,G,T for all reads at that site.

bamVI42 = pysam.AlignmentFile("VI42.trimmed.sorted.bam", "rb")

for read in bamVI42:
    for pair in read.aligned_pairs: #aligned_pairs is a tuple with where (index of read, index of reference) shows how read and reference align.
        if pair[0] is not None and pair[1] is not None: #none indicates that this portion did not map to reference, and is soft trimmed in the bamfile.
            depthVI42[pair[1]] += 1 #count that you have one read's coverage at that site
            base_composition = compositionVI42[pair[1]] #stub out empty dict that is the value associated with the composition dict key (the reference index)
            query_base = read.query_sequence[pair[0]] #what base is at this site for this read?
            if not query_base in compositionVI42[pair[1]]:
                compositionVI42[pair[1]][query_base] = 0 #add that base to the site i dict if you haven't seen the base before in any other read
            compositionVI42[pair[1]][query_base] += 1 #count the occurrence of seeing that base at site i

#note that entries will still get added if there are ANY reads at all at a site, but that the consensus sequence is masked with N's
#at any sites that have less than 20 nt of coverage. Therefore number of N's in the sequence != number of entries in the composition and depth dicts.

final_depthVI42 = dict(depthVI42)
final_compositionVI42 = dict(compositionVI42)

In [26]:
print 'There are {} sites with read pileups present in the BAM file.'.format(len(final_compositionVI42))
assert len(final_depthVI42) == len(final_compositionVI42)

There are 10374 sites with read pileups present in the BAM file.


In [27]:
with open('/Users/alliblk/Desktop/gitrepos/zika-seq/seq-validation/VI42.consensus.fasta', 'rU') as file:
    for line in file:
        if line.startswith('>'):
            continue
        else: 
            consensusVI42 = line.strip()

In [28]:
per_base_accuraciesVI42= []

for i in range(len(consensusVI42)):
    correct_base = consensusVI42[i]
    if correct_base != 'N': #don't look at sites that were masked due to coverage issues, because we don't care about sites we don't actually call
        proportion_reads_correct = float(final_compositionVI42[i][correct_base])/final_depthVI42[i]
        per_base_accuraciesVI42.append(proportion_reads_correct)
    else:
        per_base_accuraciesVI42.append(float('NaN'))
        
print "The mean per-base sequencing accuracy is {}".format(np.nanmean(per_base_accuraciesVI42))
print "The standard deviation is {}".format(np.nanstd(per_base_accuraciesVI42))
print '\n'
print "The following 0-indexed sites in the genome had sequencing accuracies less than 50%, and warrant some investigation into what's going on: {}".format([index for index,value in enumerate(per_base_accuraciesVI42) if value < 0.5])


The mean per-base sequencing accuracy is 0.96150134968
The standard deviation is 0.039551503632


The following 0-indexed sites in the genome had sequencing accuracies less than 50%, and warrant some investigation into what's going on: [1903, 5314, 5620]
